In [1]:
# Install OSMNX only if working on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install osmnx

In [1]:
# Load datasets if working on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = '...'
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])
    f = open("V2data_6mounts2022.csv.zip", "wb")
    f.write(downloaded.content.getbuffer())
    f.close()
    !unzip V2data_6mounts2022.csv.zip

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
import sys
import pickle
import osmnx as ox
import networkx as nx
import urllib, json
import requests

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    r = requests.get('https://raw.githubusercontent.com/dominik117/cortexia-darkzones-prediction/main/src/helper_scripts/data_processor.py')
    with open('data_processor.py', 'w') as f:
        f.write(r.text)
        import data_processor
    r = requests.get('https://raw.githubusercontent.com/dominik117/cortexia-darkzones-prediction/main/src/helper_scripts/darkzone_creator.py')
    with open('darkzone_creator.py', 'w') as f:
        f.write(r.text)
        import darkzone_creator
else:
    sys.path.insert(1, '../../src/')
    import helper_scripts.data_processor as data_processor

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

## Dataset Import

In [2]:
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB: df_main_url = '/content/data.csv'
else: df_main_url = '../../../Information/output/datav2.csv'
df_main = pd.read_csv(df_main_url, dtype = {'place.id': object})
df_main.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
df_main = data_processor.clean_df(df_main)
df_main = data_processor.aggregate_df(df_main)
df_main['date_utc'] = df_main['date_utc'].astype(str)
df_main['row_type'] = "ground truth"

In [3]:
# Load datasets if working on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = '190i18Wz2-HBXa82iajuNEPpCdFaOVL2r'
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])
    f = open("darkzones_predicted.csv", "wb")
    f.write(downloaded.content.getbuffer())
    f.close()

if IN_COLAB: df_darkzones_url = '/content/darkzones_predicted.csv'
else: df_darkzones_url = '../../src/data/darkzones_predicted.csv'
df_darkzones = pd.read_csv(df_darkzones_url)
df_darkzones.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
df_darkzones.drop('predicted_total', axis=1, inplace=True, errors='ignore')
df_darkzones.rename(columns = {'actual_total':'total_litter'}, inplace = True)

In [4]:
df = pd.concat([df_main, df_darkzones], ignore_index=True)

# Mapping

In [9]:
def clean_coordinates(coordinates):
    new = []
    for pair in coordinates:
        new.append((pair[1], pair[0]))
    return new
with urllib.request.urlopen('https://raw.githubusercontent.com/dominik117/data-science-toolkit/main/data/edges.geojson') as url:
    data = json.loads(url.read().decode())
df_edges = pd.DataFrame(data['features'])  # <-- The only column needed
df_edges.rename(columns = {'id':'edge_id'}, inplace = True)
df_edges = pd.concat([df_edges.drop(['geometry'], axis=1), df_edges['geometry'].apply(pd.Series)], axis=1)  # <--Explode dictionary inside cells
df_edges = pd.concat([df_edges.drop(['properties'], axis=1), df_edges['properties'].apply(pd.Series)], axis=1)  # <--Explode dictionary inside cells
df_edges = df_edges[['edge_id', 'coordinates', 'name', 'length']]
df_edges['coordinates'] = df_edges['coordinates'].apply(clean_coordinates)
df = pd.merge(df, df_edges[['edge_id', 'length']], how="left", on="edge_id")

In [6]:
north = 47.574840  #47.569840
south = 47.559453  #47.565453
west = 7.579114  #7.585114
east = 7.609701  #7.594701

route = []
for index, row in df_edges.iterrows():
    for pair in row['coordinates']:
        if (pair[0] < north) & (pair[0] > south) & (pair[1] > west) & (pair[1] < east):
            route.append(row['edge_id'])
route_set = set(route)
route = []
for item in route_set:
    route.append(item)

In [ ]:
import folium

df_map = df[(df['date_utc'] == '2022-06-24') & (df['length'] > 25)]

df_map_route = df_map.copy()
df_map_route = df_map[pd.DataFrame(df_map['edge_id'].tolist()).isin(route).any(1).values]

df_map_ground = df_map_route[(df_map_route['row_type'] == "ground truth")]
df_map_darkzone = df_map_route[(df_map_route['row_type'] == "darkzone")]
df_map.shape

# cartodbdark_matter, cartodbpositron
m = folium.Map(location=[47.5596, 7.5886], zoom_start=13.5, prefer_canvas=True, tiles="cartodbpositron")

for index, row in df_map_ground.iterrows():
        loc = df_edges['coordinates'][df_edges['edge_id'] == row['edge_id']].tolist()
        loc = loc[0]
        folium.PolyLine(loc, color='seagreen',weight=2,opacity=1).add_to(m)  #3e75bd #00bbf9 #049ed1
for index, row in df_map_darkzone.iterrows():
        loc = df_edges['coordinates'][df_edges['edge_id'] == row['edge_id']].tolist()
        loc = loc[0]
        folium.PolyLine(loc, color='seagreen',weight=2,opacity=1).add_to(m)   #fee440  #ebc810
# royalblue, seagreen, indianred, #087E8B
m

In [34]:
m.save('map.html')